# import libraries

In [16]:
from environment.mountain_car import MountainCarEnv #as MountainCarEnv

from environment.algorithms_planner import Planner as Planner_2   

from environment.algorithms_rl import RL as RL_2  
import numpy as np
import matplotlib.pyplot as plt

import pickle
import gym
from environment.test_env import TestEnv_2 as TestEnv_2
import environment.multi_mc_2 as ml5
from itertools import repeat, product
import gc
import pandas as pd
import json

In [3]:
MountainCarEnv_ = MountainCarEnv(dim=5,base =False,max_episode_steps=200)

# Function definitions

## create n mc | environment instances

In [25]:

def create_n_mc(map_seed = 1, count =10,
                env = MountainCarEnv,gym_configurations = {'id' : 'Frozenlake-v1_cust', 
                                                   'render_mode':None},
                 generate_random_map_config = dict(p =0.9),
                 size = 5,
                multiplier = 1000,
               
               base=False,
               max_episode_steps = 200,
                
                
                
                
               
               ):
    
    np.random.seed(map_seed)
    #random.seed(map_seed)
    


    env_list = []
    
    seed_list = [ map_seed+i*multiplier for i in range(count)]
    
    for i in seed_list:
        env_temp = env(dim=size,
                                    base =base,
                                      max_episode_steps=max_episode_steps)

        env_temp.env.reset(seed =i)

        env_list.append(env_temp)        

        
        
        
        
        
    return seed_list,env_list

## function definition save q learning training data

In [2]:


def save_Q_itteration_data(test_pik,env_name = 'MC_',iter_ = 'RL_iter',
                              
                              name = 'test_data1', window =1
                              
                              
                              ):
    
    
    base_address =r'./ADA/'+env_name+'/'+iter_+'/'

    print(base_address,name)
    
    
    for i in range(len(test_pik['response']['train_responses'])):


        gk = pd.DataFrame(#test_pik['response']['train_responses'][i][1]


          test_pik['response']['train_responses'][i][3].max(axis=2).astype('float16')#[0:test_pik['response']['train_responses'][i][3].shape[0]:window,:] #.mean(1)


        )


        gk['trial'] = i

        
        
        
        gk.to_pickle(base_address+'V_track/'+name+'_'+str(i)+'.gzip',

            #partition_cols = 'trial',
        compression ='gzip', #'brotli',#


            )




        gk = pd.DataFrame(np.array(test_pik['response']['train_responses'][i][4]))
        gk['trial'] = i



        gk.astype('int').to_pickle(base_address+'/PI_track/'+name+'_'+str(i)+'.gzip',

            #partition_cols = 'trial',
        compression ='gzip', #'brotli',#


            )








    kl =pd.DataFrame.from_records([test_pik['response']['train_responses'][q][2]



                               for q in range(len(test_pik['response']['train_responses']))


                              ])


    kl['optima'] = [test_pik['response']['train_responses'][q][5] for q in range(len(test_pik['response']['train_responses']))]


    kl.to_pickle(base_address+'other_info/P/'+name+'.gzip',
                
                compression ='gzip'
                )




    kl =pd.DataFrame.from_records([test_pik['response']['train_responses'][q][1]



                               for q in range(len(test_pik['response']['train_responses']))


                              ])


    kl['optima'] = [test_pik['response']['train_responses'][q][5] for q in range(len(test_pik['response']['train_responses']))]

    kl.astype('float32').to_pickle(base_address+'other_info/V/'+name+'.gzip',
                                  
                                  compression ='gzip'
                                  )






    kl = pd.Series([test_pik['response']['train_responses'][q][0].astype('float16') for q in range(len(test_pik['response']['train_responses']))

    ])
    kl.to_pickle(base_address+'other_info/Q/'+name+'.gzip',compression ='gzip')




    with open(base_address+'other_info/config/'+name+'.json', 'w', encoding='utf-8') as f:
        json.dump({i:test_pik[i] for i in ['config', 'VI', 'gamma', 'init_alpha'
,'init_epsilon' ,'min_alpha', 'size']} 
                  , f, ensure_ascii=False, indent=4)
        
        
        
        
        



In [3]:

def _convert_state_obs_method(state,done,package_select=None):




    return (-1 if done else mapper_state(
                                discretize_state(state,
                                         package_select['low'],
                                        package_select['bin_width'],
                                        package_select['state_space']),
                                package_select['seriaized']))




## functon definition : training over grid Q learing 

In [4]:
%%time

sizes_rid = [5,]#8,12,16,20,24,28,32]#[4,8,12,16,20]

grid = {

'gamma' : [0.01],#0.3],#0.7,0.9,0.95,0.99,1],
'init_alpha' : [ 0.95],#0.9],#0.7,0.4 ],
'init_epsilon' : [ 1],#0.95],#0.9,0.7,0.4 ]
    
}

misc = 'QL_MC'
vi = True


env_name ='MC'
iter_ = 'RL_iter'




PROCESSES=8
type_ = 'starmap_async'
n_iters=5000#00
gym_configurations = {'id' : 'Frozenlake-v1_cust','render_mode':None}
map_seed = 1
count =10
generate_random_map_config = dict(p =0.9)
multiplier = 1000
min_alpha=0.001
window=100
threshold = 0.01
max_mul = 5


#map_list =[], env_list = [],

create_new_map_flag = True


base =False
max_episode_steps=200





def run_Q_model_over_grid(grid =grid
                              ,sizes_rid =sizes_rid
                              ,misc =misc
                              ,vi=vi
                              ,env_name ='FL',iter_ = 'RL_iter',
                             version = '1'
                              
                              ,PROCESSES = PROCESSES
                                ,type_ = type_
                                ,n_iters = n_iters
                                ,gym_configurations = gym_configurations
                                ,map_seed = map_seed
                                ,count = count
                                ,generate_random_map_config = generate_random_map_config
                                ,multiplier = multiplier
                             ,create_new_map_flag = create_new_map_flag
                            ,min_alpha=min_alpha
                            ,window=window
                            ,threshold = threshold
                            ,max_mul=max_mul
                            ,base = base
                            ,max_episode_steps = max_episode_steps



                             ):


    #grid_response = dict({})




    packages = pickle.load(open('environment/MountainCar_envP_config.pkl', "rb"))




    address_dict = []

    for o in sizes_rid:

        
        
        
        
        
        
        
        

        package_select = packages[o]
        
        #global _convert_state_obs_method
        

        

        
        
        
        
        if create_new_map_flag :#and (len(map_list)!=count):
            map_list,env_list = ml5.create_n_mc(
                                    map_seed = map_seed,
                                    count =count,

                                    env = MountainCarEnv,
                                    gym_configurations = gym_configurations,
                                     generate_random_map_config = generate_random_map_config,
                                     size = o,#4,
                                     multiplier = multiplier
                                        ,base = base
                                        ,max_episode_steps = max_episode_steps
                        
                
                
                
                
                                            )

            
        base_address =r'./ADA/'+env_name+'/'+iter_+'/'
        name = misc+"_size_"+str(o)

        jl = pd.DataFrame(map_list)    

        #jl.columns = jl.columns.astype(str)

        #jl.to_parquet(r'./ADA/FL/Valu_iter/map_list/_test_data1.gzip',
        #
        #    #partition_cols = 'trial',
        #compression ='gzip', #'brotli',#
        #             )   

        jl.to_pickle(base_address+'map_list/'+name+'.gzip',

            #partition_cols = 'trial',
        compression ='gzip', #'brotli',#
                     )  





        jl = pd.DataFrame(env_list)  

        #jl.columns = jl.columns.astype(str)

        jl.to_pickle(base_address+'env_list/'+name+'.gzip',

            #partition_cols = 'trial',
        compression ='gzip', #'brotli',#
                     )           
        
        
        
        
        
        
        
        
        

        for k,(i,j,m) in enumerate(product(grid['gamma'],grid['init_alpha'],
                                    
                                    grid['init_epsilon']
                                    )):

            
            
            

            responses_vi= ml5.main_rl_mc(
                        package_select=package_select,

                      driver_func_config = dict(PROCESSES=PROCESSES,type_ = type_),

                      learning_function_config = dict(kwarg = dict(   
                                                  n_episodes=n_iters,#100000,

                                                 gamma = i,
                                                 init_alpha = j ,
                                                 init_epsilon =m ,

                                                min_alpha=min_alpha,
                                                  #convert_state_obs = _convert_state_obs_method,
                          
                          
                                                  window=window,threshold = threshold,
                                                  max_mul=max_mul
                          


                                                )),

                     count_map_config =dict(map_seed = map_seed,
                                            count =count,

                                            env = MountainCarEnv,
                                            gym_configurations = gym_configurations,
                                             generate_random_map_config = generate_random_map_config,
                                             size = o,#4,
                                             multiplier = multiplier,

                                            base =False,
                                            max_episode_steps=200


                                           ),
                                        
                                        
                                        
                                        map_list=map_list,env_list=env_list,
                                        create_new_map_flag = create_new_map_flag
                                       
                                       )




            config = dict(driver_func_config = dict(PROCESSES=PROCESSES,type_ = type_),

                      learning_function_config = dict(kwarg = dict(   
                                                  n_episodes=n_iters,#100000,

                                                 gamma = i,
                                                 init_alpha = j ,
                                                 init_epsilon =m ,

                                                min_alpha=min_alpha,
                          
                          
                                                  window=window,threshold = threshold,
                                                  max_mul=max_mul
                          


                                                )),

                     count_map_config =dict(map_seed = map_seed,
                                            count =count,

                                            #env = MountainCarEnv,
                                            gym_configurations = gym_configurations,
                                             generate_random_map_config = generate_random_map_config,
                                             size = o,#4,
                                             multiplier = multiplier,

                                            base =False,
                                            max_episode_steps=200


                                           ),
                                        
                                        
                                        
                                        #map_list=map_list,env_list=env_list,
                                        create_new_map_flag = create_new_map_flag
                                       )

            pickle_file = {'response':responses_vi,'config':config,'VI':True,

                           'gamma':i,
                           'init_alpha' :j ,
                           'init_epsilon' :m ,
                           'size' : o,
                           'min_alpha':min_alpha,



                          }



            

            name = misc+"_size_"+str(o)+"_gamma_"+str(i)+'_alpha_'+str(j)+'_eps_'+str(m)

            save_Q_itteration_data(pickle_file,
                                       env_name =env_name,
                                       iter_ = iter_,
            
            name = name


                            )
        
            del responses_vi
            del pickle_file
            
            #with open(r"./ADA/"+misc+"_size_"+str(o)+"_gamma_"+str(i)+'_theta_'+str(j)+".pickle", "wb") as fi:
            #    pickle.dump(pickle_file,fi)


            
            address_dict.append({'size':o,'gamma':i,'theta':j,'index':k,
                                 'address_name':name, 'misc':misc,

                                'iter_' :iter_,
                                 'env_name':env_name

                                })
            
            gc.collect()

            
            
    
    
    address_df = pd.DataFrame(address_dict) 
    
    address_df.to_csv(misc+env_name+iter_+version+'.csv')
    
    
    return pd.DataFrame(address_dict)  #, pickle_file         
               
        

CPU times: user 10 µs, sys: 1 µs, total: 11 µs
Wall time: 13.8 µs


## function definition save VI-PI training data

In [ ]:


def read_and_save_vi_pi_itteration_data( grid,sizes_rid,misc,env_name ='FL',iter_ = 'Valu_iter'


                                        ):

    
    address_dict = []
    

    for o in sizes_rid:


        for k,(i,j) in enumerate(product(grid['gamma'],grid['theta'])):


            with open(r"./ADA/"+misc+"_size_"+str(o)+"_gamma_"+str(i)+'_theta_'+str(j)+".pickle", "rb") as fi:
                        test_pik = pickle.load(fi)
                    
            name = misc+"_size_"+str(o)+"_gamma_"+str(i)+'_theta_'+str(j)
            
            save_vi_pi_itteration_data(test_pik,env_name =env_name,iter_ = iter_,
                              
                              name = name
                              
                              
                              )
            
            address_dict.append({'size':o,'gamma':i,'theta':j,'index':k,
                                 'address_name':name, 'misc':misc,
                                
                                'iter_' :iter_,
                                 'env_name':env_name
                                 
                                })
            
    return pd.DataFrame(address_dict)








def save_vi_pi_itteration_data(test_pik,env_name = 'FL_',iter_ = 'Valu_iter',
                              
                              name = 'test_data1'
                              
                              
                              ):
    
    
    base_address =r'./ADA/'+env_name+'/'+iter_+'/'

    print(base_address,name)
    
    
    for i in range(len(test_pik['response']['train_responses'])):
        gk = pd.DataFrame(test_pik['response']['train_responses'][i][1])
        gk['trial'] = i

        
        gk.astype('float32').to_pickle(base_address+'V_track/'+name+'_'+str(i)+'.gzip',

            #partition_cols = 'trial',
        compression ='gzip', #'brotli',#


            )




        gk = pd.DataFrame(test_pik['response']['train_responses'][i][3])
        gk['trial'] = i




        gk.astype('float32').to_pickle(base_address+'/PI_track/'+name+'_'+str(i)+'.gzip',

            #partition_cols = 'trial',
        compression ='gzip', #'brotli',#


            )








    kl =pd.DataFrame.from_records([test_pik['response']['train_responses'][q][2]



                               for q in range(len(test_pik['response']['train_responses']))


                              ])


    kl['optima'] = [test_pik['response']['train_responses'][q][4] for q in range(len(test_pik['response']['train_responses']))]



    kl.to_pickle(base_address+'other_info/P/'+name+'.gzip',
                
                compression ='gzip'
                )




    kl =pd.DataFrame.from_records([test_pik['response']['train_responses'][q][0]



                               for q in range(len(test_pik['response']['train_responses']))


                              ])


    kl['optima'] = [test_pik['response']['train_responses'][q][4] for q in range(kl.shape[0])]

    kl.astype('float32').to_pickle(base_address+'other_info/V/'+name+'.gzip',
                                  
                                  compression ='gzip'
                                  )



    with open(base_address+'other_info/config/'+name+'.json', 'w', encoding='utf-8') as f:
        json.dump({i:test_pik[i] for i in ['config', 'VI', 'gamma', 'theta', 'size']} 
                  , f, ensure_ascii=False, indent=4)
        
        
        
        
        



## functon definition : training over grid PI VI

In [ ]:
%%time

sizes_rid = [5,] #[5,10,15,20,25,30]#[8,12,16,20]#[4,8,12,16,20]

grid = {

'gamma' : [0.01,0.1,0.3,0.5,0.7,0.9,0.95,0.99,1],
'theta' : [ 1e-13 ]


    
    
}

misc = 'VI_FL'
vi = True


env_name ='FL'
iter_ = 'Valu_iter'







PROCESSES=8
type_ = 'starmap_async'
n_iters=1000#00
gym_configurations = {'id' : 'MountainCarEnv-v1_cust','render_mode':None}
map_seed = 1
count =5#10





base =False
max_episode_steps=2000 #2000





generate_random_map_config = dict(p =0.9)
multiplier = 1000





#map_list =[], env_list = [],

create_new_map_flag = True





def run_VI_PI_model_over_grid_mc(grid =grid
                              ,sizes_rid =sizes_rid
                              ,misc =misc
                              ,vi=vi
                              ,env_name ='MC',iter_ = 'Valu_iter',
                             version = '1'
                              
                              ,PROCESSES = PROCESSES
                                ,type_ = type_
                                ,n_iters = n_iters
                                ,gym_configurations = gym_configurations
                                ,map_seed = map_seed
                                ,count = count
                                ,generate_random_map_config = generate_random_map_config
                                ,multiplier = multiplier
                             ,create_new_map_flag = create_new_map_flag
                              ,base = base
                                ,max_episode_steps = max_episode_steps
                              
                             ):


    #grid_response = dict({})

    address_dict = []

    for o in sizes_rid:

        
        
        if create_new_map_flag :#and (len(map_list)!=count):
            map_list,env_list = ml5.create_n_mc(
                                    map_seed = map_seed,
                                    count =count,

                                    env = MountainCarEnv,
                                    gym_configurations = gym_configurations,
                                     generate_random_map_config = generate_random_map_config,
                                     size = o,#4,
                                     multiplier = multiplier
                                        ,base = base
                                        ,max_episode_steps = max_episode_steps
                        
                
                
                
                
                                            )
            
            
            
            
            
        base_address =r'./ADA/'+env_name+'/'+iter_+'/'
        name = misc+"_size_"+str(o)

        jl = pd.DataFrame(map_list)    

        #jl.columns = jl.columns.astype(str)

        #jl.to_parquet(r'./ADA/FL/Valu_iter/map_list/_test_data1.gzip',
        #
        #    #partition_cols = 'trial',
        #compression ='gzip', #'brotli',#
        #             )   

        jl.to_pickle(base_address+'map_list/'+name+'.gzip',

            #partition_cols = 'trial',
        compression ='gzip', #'brotli',#
                     )  





        jl = pd.DataFrame(env_list)  

        #jl.columns = jl.columns.astype(str)

        jl.to_pickle(base_address+'env_list/'+name+'.gzip',

            #partition_cols = 'trial',
        compression ='gzip', #'brotli',#
                     )                

            
        
        

        for k,(i,j) in enumerate(product(grid['gamma'],grid['theta'])):
            
            
            
            
            responses_vi= ml5.main_pivi_mc(driver_func_config = dict(PROCESSES=PROCESSES,type_ = type_),

                      learning_function_config = dict(kwarg = dict(n_iters=n_iters,#100000,

                                                                   gamma = i,#0.1,#0.99,
                                                                   theta = j   



                                                                  ),vi =vi),
                     count_map_config =dict(map_seed = map_seed,
                                                        count =count,

                                                        env = MountainCarEnv,
                                                        gym_configurations = gym_configurations,
                                                         generate_random_map_config = generate_random_map_config,
                                                         size = o,#4,
                                                         multiplier = multiplier
                                                            ,base = base
                                                            ,max_episode_steps = max_episode_steps


                
                                           ),
                                        
                                        
                                        
                                        map_list=map_list,env_list=env_list,
                                        create_new_map_flag = create_new_map_flag
                                       
                                       
                                       )
            
            
            
            
            
            
            

            #grid_response[k] = dict(
            #                    gamma=i,
            #                    theta =j ,
            #                    response = responses_vi
            #
            #                    )
            config = dict(driver_func_config = dict(PROCESSES=PROCESSES,type_ = type_),

                      learning_function_config = dict(kwarg = dict(n_iters=n_iters,#100000,

                                                                   gamma = i,#0.1,#0.99,
                                                                   theta = j   



                                                                  ),vi =vi),
                     count_map_config =dict(map_seed = map_seed,
                                                        count =count,

                                                        #env = MountainCarEnv,
                                                        gym_configurations = gym_configurations,
                                                         generate_random_map_config = generate_random_map_config,
                                                         size = o,#4,
                                                         multiplier = multiplier
                                                            ,base = base
                                                            ,max_episode_steps = max_episode_steps


                
                                           ),
                                        
                                        
                                        
                                        #map_list=map_list,env_list=env_list,
                                        #create_new_map_flag = create_new_map_flag
                                       )

            
            
            
            
            
            print('here')
            
            pickle_file = {'response':responses_vi,'config':config,'VI':True,

                           'gamma':i,
                           'theta' :j ,
                           'size' : o
                          }



            

            name = misc+"_size_"+str(o)+"_gamma_"+str(i)+'_theta_'+str(j)

            save_vi_pi_itteration_data(pickle_file,
                                       env_name =env_name,
                                       iter_ = iter_,

                              name = name


                              )
            
            del responses_vi
            del pickle_file
            
            #with open(r"./ADA/"+misc+"_size_"+str(o)+"_gamma_"+str(i)+'_theta_'+str(j)+".pickle", "wb") as fi:
            #    pickle.dump(pickle_file,fi)


            
            address_dict.append({'size':o,'gamma':i,'theta':j,'index':k,
                                 'address_name':name, 'misc':misc,

                                'iter_' :iter_,
                                 'env_name':env_name

                                })
            
            gc.collect()

            
            
    
    
    address_df = pd.DataFrame(address_dict) 
    
    address_df.to_csv(misc+env_name+iter_+version+'.csv')
    
    
    return pd.DataFrame(address_dict)   #,pickle_file        
        
        
        
        
        


        
        
        
        
        

        
        
        
        



# Generating training instances and training over grid

## Q learning | Selected size for | mountain car : 25

In [11]:
%%time

sizes_rid = [25]#[5,10,15,20,25,30]#[4,8,12,16,20]

grid = {

'gamma' : [0.01,0.3,0.7,0.9,0.95,0.99,],#1],
'init_alpha' : [ 0.95,0.9,0.7,0.4 ],
'init_epsilon' : [ 1,0.95,0.9,0.7,0.4 ]
    
}

misc = 'QL_MC'
vi = True


env_name ='MC'
iter_ = 'RL_iter'

version = '10'


PROCESSES=8
type_ = 'starmap_async'
n_iters=1000#0
gym_configurations = {'id' : 'MountainCarEnv-v1_cust','render_mode':None}
map_seed = 1
count =5#10
generate_random_map_config = dict(p =0.9)
multiplier = 1000
min_alpha=0.001
window=10#0
threshold = 0.01
max_mul = 5


#map_list =[], env_list = [],

create_new_map_flag = True




base =False
max_episode_steps=2000





run_Q_model_over_grid(grid =grid
                              ,sizes_rid =sizes_rid
                              ,misc =misc
                              ,vi=vi
                              ,env_name =env_name,iter_ = iter_,
                             version = version
                              
                              ,PROCESSES = PROCESSES
                                ,type_ = type_
                                ,n_iters = n_iters
                                ,gym_configurations = gym_configurations
                                ,map_seed = map_seed
                                ,count = count
                                ,generate_random_map_config = generate_random_map_config
                                ,multiplier = multiplier
                             ,create_new_map_flag = create_new_map_flag
                            ,min_alpha=min_alpha
                            ,window=window
                            ,threshold = threshold
                            ,max_mul=max_mul
                            ,base = base
                            ,max_episode_steps = max_episode_steps



                             )

/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/Documents/OMSCS/CS7641/Assignment_4/assignment4_git/environment/algorithms_rl.py:254: UserWarning: Episode was truncated.  Bootstrapping 0 reward.
  warnings.warn("Episode was truncated.  Bootstrapping 0 reward.")
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (boo

runtime = 10.34 seconds
runtime = 10.62 seconds
runtime = 20.28 seconds
runtime = 17.14 seconds
./ADA/MC/RL_iter/ QL_MC_size_25_gamma_0.01_alpha_0.95_eps_1


/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/Documents/OMSCS/CS7641/Assignment_4/assignment4_git/environment/algorithms_rl.py:254: UserWarning: Episode was truncated.  Bootstrapping 0 reward.
  warnings.warn("Episode was truncated.  Bootstrapping 0 reward.")
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (boo

runtime = 18.69 seconds
runtime = 21.37 seconds
runtime = 25.67 seconds
runtime = 27.15 seconds
runtime = 27.67 seconds
./ADA/MC/RL_iter/ QL_MC_size_25_gamma_0.01_alpha_0.95_eps_0.95


/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/Documents/OMSCS/CS7641/Assignment_4/assignment4_git/environment/algorithms_rl.py:254: UserWarning: Episode was truncated.  Bootstrapping 0 reward.
  warnings.warn("Episode was truncated.  Bootstrapping 0 reward.")
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (boo

...

/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/Documents/OMSCS/CS7641/Assignment_4/assignment4_git/environment/algorithms_rl.py:254: UserWarning: Episode was truncated.  Bootstrapping 0 reward.
  warnings.warn("Episode was truncated.  Bootstrapping 0 reward.")
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (boo

runtime = 13.68 seconds
runtime = 18.35 seconds
runtime = 18.93 seconds
runtime = 22.72 seconds
runtime = 22.61 seconds
./ADA/MC/RL_iter/ QL_MC_size_25_gamma_0.99_alpha_0.4_eps_0.7


/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/Documents/OMSCS/CS7641/Assignment_4/assignment4_git/environment/algorithms_rl.py:254: UserWarning: Episode was truncated.  Bootstrapping 0 reward.
  warnings.warn("Episode was truncated.  Bootstrapping 0 reward.")
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (boo

runtime = 12.03 seconds
runtime = 13.35 seconds
runtime = 13.74 seconds
runtime = 21.18 seconds
runtime = 33.62 seconds
./ADA/MC/RL_iter/ QL_MC_size_25_gamma_0.99_alpha_0.4_eps_0.4
CPU times: user 2min 28s, sys: 35.4 s, total: 3min 3s
Wall time: 56min 52s


,size,gamma,theta,index,address_name,misc,iter_,env_name
0,25,0.01,0.95,0,QL_MC_size_25_gamma_0.01_alpha_0.95_eps_1,QL_MC,RL_iter,MC
1,25,0.01,0.95,1,QL_MC_size_25_gamma_0.01_alpha_0.95_eps_0.95,QL_MC,RL_iter,MC
2,25,0.01,0.95,2,QL_MC_size_25_gamma_0.01_alpha_0.95_eps_0.9,QL_MC,RL_iter,MC
3,25,0.01,0.95,3,QL_MC_size_25_gamma_0.01_alpha_0.95_eps_0.7,QL_MC,RL_iter,MC
4,25,0.01,0.95,4,QL_MC_size_25_gamma_0.01_alpha_0.95_eps_0.4,QL_MC,RL_iter,MC
...,...,...,...,...,...,...,...,...
115,25,0.99,0.40,115,QL_MC_size_25_gamma_0.99_alpha_0.4_eps_1,QL_MC,RL_iter,MC
116,25,0.99,0.40,116,QL_MC_size_25_gamma_0.99_alpha_0.4_eps_0.95,QL_MC,RL_iter,MC
117,25,0.99,0.40,117,QL_MC_size_25_gamma_0.99_alpha_0.4_eps_0.9,QL_MC,RL_iter,MC
118,25,0.99,0.40,118,QL_MC_size_25_gamma_0.99_alpha_0.4_eps_0.7,QL_MC,RL_iter,MC


## Q learning | mountain car | comparitive analysis for other size

In [14]:
%%time

sizes_rid = [5,10,15,20,30]#[4,8,12,16,20]

grid = {

'gamma' : [0.7,0.9],#[0.01,0.3,0.7,0.9,0.95,0.99,],#1],
'init_alpha' : [0.95],#[ 0.95,0.9,0.7,0.4 ],
'init_epsilon' : [0.9]#[ 1,0.95,0.9,0.7,0.4 ]
    
}

misc = 'QL_MC'
vi = True


env_name ='MC'
iter_ = 'RL_iter'

version = '2'


PROCESSES=8
type_ = 'starmap_async'
n_iters=1000#0
gym_configurations = {'id' : 'MountainCarEnv-v1_cust','render_mode':None}
map_seed = 1
count =5#10
generate_random_map_config = dict(p =0.9)
multiplier = 1000
min_alpha=0.001
window=10#0
threshold = 0.01
max_mul = 5


#map_list =[], env_list = [],

create_new_map_flag = True




base =False
max_episode_steps=2000





run_Q_model_over_grid(grid =grid
                              ,sizes_rid =sizes_rid
                              ,misc =misc
                              ,vi=vi
                              ,env_name =env_name,iter_ = iter_,
                             version = version
                              
                              ,PROCESSES = PROCESSES
                                ,type_ = type_
                                ,n_iters = n_iters
                                ,gym_configurations = gym_configurations
                                ,map_seed = map_seed
                                ,count = count
                                ,generate_random_map_config = generate_random_map_config
                                ,multiplier = multiplier
                             ,create_new_map_flag = create_new_map_flag
                            ,min_alpha=min_alpha
                            ,window=window
                            ,threshold = threshold
                            ,max_mul=max_mul
                            ,base = base
                            ,max_episode_steps = max_episode_steps



                             )

/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/envs/registration.py:498: UserWarning: WARN: Overriding environment MountainCarEnv-v1_cust already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/Documents/OMSCS/CS7641/Assignment_4/assignment4_git/environment/algorithms_rl.py:254: UserWarning: Episode was truncated.  Bootstrapping 0 reward.
  warnings.warn("Episode was truncated.  Bootstrapping 0 reward.")
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(termina

runtime = 24.64 seconds
runtime = 27.85 seconds
runtime = 28.41 seconds
runtime = 27.20 seconds
./ADA/MC/RL_iter/ QL_MC_size_5_gamma_0.7_alpha_0.95_eps_0.9


/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/Documents/OMSCS/CS7641/Assignment_4/assignment4_git/environment/algorithms_rl.py:254: UserWarning: Episode was truncated.  Bootstrapping 0 reward.
  warnings.warn("Episode was truncated.  Bootstrapping 0 reward.")
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/Documents/OMSCS/CS7641/Assignment_4/assignment4_git/environment/algorithms_rl.py:254: UserWarning: Episode was truncated.  Bootstrapping 0 reward.
  warnings.warn("Episode was truncated.  Bootstrapping 0 reward.")
/Users/ash

runtime = 23.01 seconds
runtime = 23.40 seconds
runtime = 23.41 seconds
runtime = 25.06 seconds
./ADA/MC/RL_iter/ QL_MC_size_5_gamma_0.9_alpha_0.95_eps_0.9


/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/envs/registration.py:498: UserWarning: WARN: Overriding environment MountainCarEnv-v1_cust already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/Documents/OMSCS/CS7641/Assignment_4/assignment4_git/environment/algorithms_rl.py:254: UserWarning: Episode was truncated.  Bootstrapping 0 reward.
  warnings.warn("Episode was truncated.  Bootstrapping 0 reward.")
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(termina

...

/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/envs/registration.py:498: UserWarning: WARN: Overriding environment MountainCarEnv-v1_cust already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/Documents/OMSCS/CS7641/Assignment_4/assignment4_git/environment/algorithms_rl.py:254: UserWarning: Episode was truncated.  Bootstrapping 0 reward.
  warnings.warn("Episode was truncated.  Bootstrapping 0 reward.")
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(termina

runtime = 18.16 seconds
runtime = 18.36 seconds
runtime = 19.63 seconds
runtime = 19.00 seconds
runtime = 18.93 seconds
./ADA/MC/RL_iter/ QL_MC_size_30_gamma_0.7_alpha_0.95_eps_0.9


/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/Documents/OMSCS/CS7641/Assignment_4/assignment4_git/environment/algorithms_rl.py:254: UserWarning: Episode was truncated.  Bootstrapping 0 reward.
  warnings.warn("Episode was truncated.  Bootstrapping 0 reward.")
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (boo

runtime = 19.68 seconds
runtime = 19.92 seconds
runtime = 20.26 seconds
runtime = 20.20 seconds
runtime = 20.72 seconds
./ADA/MC/RL_iter/ QL_MC_size_30_gamma_0.9_alpha_0.95_eps_0.9
CPU times: user 15.7 s, sys: 2.23 s, total: 17.9 s
Wall time: 4min 24s


,size,gamma,theta,index,address_name,misc,iter_,env_name
0,5,0.7,0.95,0,QL_MC_size_5_gamma_0.7_alpha_0.95_eps_0.9,QL_MC,RL_iter,MC
1,5,0.9,0.95,1,QL_MC_size_5_gamma_0.9_alpha_0.95_eps_0.9,QL_MC,RL_iter,MC
2,10,0.7,0.95,0,QL_MC_size_10_gamma_0.7_alpha_0.95_eps_0.9,QL_MC,RL_iter,MC
3,10,0.9,0.95,1,QL_MC_size_10_gamma_0.9_alpha_0.95_eps_0.9,QL_MC,RL_iter,MC
4,15,0.7,0.95,0,QL_MC_size_15_gamma_0.7_alpha_0.95_eps_0.9,QL_MC,RL_iter,MC
5,15,0.9,0.95,1,QL_MC_size_15_gamma_0.9_alpha_0.95_eps_0.9,QL_MC,RL_iter,MC
6,20,0.7,0.95,0,QL_MC_size_20_gamma_0.7_alpha_0.95_eps_0.9,QL_MC,RL_iter,MC
7,20,0.9,0.95,1,QL_MC_size_20_gamma_0.9_alpha_0.95_eps_0.9,QL_MC,RL_iter,MC
8,30,0.7,0.95,0,QL_MC_size_30_gamma_0.7_alpha_0.95_eps_0.9,QL_MC,RL_iter,MC
9,30,0.9,0.95,1,QL_MC_size_30_gamma_0.9_alpha_0.95_eps_0.9,QL_MC,RL_iter,MC


## Value Iterations | all sizes | mountain car 

In [35]:

sizes_rid =  [5,10,15,20,25,30]#[5#,#10,15,20,25,30]#[8,12,16,20]#[4,8,12,16,20]

grid = {

'gamma' : [0.01,0.1,0.3,0.5,0.7,0.9,0.95,0.99],#1],
'theta' : [ 1e-13 ]


    
    
}

misc = 'VI_MC'
vi = True


env_name ='MC'
iter_ = 'Valu_iter'



PROCESSES=8
type_ = 'starmap_async'
n_iters=1000#00
gym_configurations = {'id' : 'MountainCarEnv-v1_cust','render_mode':None}
map_seed = 1
count =5#10

version = '1'



base =False
max_episode_steps=2000#200





generate_random_map_config = dict(p =0.9)
multiplier = 1000





#map_list =[], env_list = [],

create_new_map_flag = True




resp_ = run_VI_PI_model_over_grid_mc(grid =grid
                              ,sizes_rid =sizes_rid
                              ,misc =misc
                              ,vi=vi
                              ,env_name =env_name,iter_ = iter_,
                             version = version
                              
                              ,PROCESSES = PROCESSES
                                ,type_ = type_
                                ,n_iters = n_iters
                                ,gym_configurations = gym_configurations
                                ,map_seed = map_seed
                                ,count = count
                                ,generate_random_map_config = generate_random_map_config
                                ,multiplier = multiplier
                             ,create_new_map_flag = create_new_map_flag
                              ,base = base
                                ,max_episode_steps = max_episode_steps
                              
                             )






/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/envs/registration.py:498: UserWarning: WARN: Overriding environment MountainCarEnv-v1_cust already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


runtime = 0.00 seconds
runtime = 0.03 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
here
./ADA/MC/Valu_iter/ VI_MC_size_5_gamma_0.01_theta_1e-13
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
here
./ADA/MC/Valu_iter/ VI_MC_size_5_gamma_0.1_theta_1e-13
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
here
./ADA/MC/Valu_iter/ VI_MC_size_5_gamma_0.3_theta_1e-13
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.02 seconds
runtime = 0.09 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
here
./ADA/MC/

/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/envs/registration.py:498: UserWarning: WARN: Overriding environment MountainCarEnv-v1_cust already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


runtime = 0.03 seconds
runtime = 0.03 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
here
./ADA/MC/Valu_iter/ VI_MC_size_10_gamma_0.01_theta_1e-13
runtime = 0.01 seconds
runtime = 0.01 seconds
runtime = 0.01 seconds
runtime = 0.02 seconds
runtime = 0.01 seconds
runtime = 0.01 seconds
runtime = 0.01 seconds
runtime = 0.01 seconds
runtime = 0.01 seconds
here
./ADA/MC/Valu_iter/ VI_MC_size_10_gamma_0.1_theta_1e-13
runtime = 0.01 seconds
runtime = 0.10 seconds
runtime = 0.01 seconds
runtime = 0.02 seconds
runtime = 0.02 seconds
runtime = 0.01 seconds
runtime = 0.01 seconds
runtime = 0.01 seconds
runtime = 0.01 seconds
here
./ADA/MC/Valu_iter/ VI_MC_size_10_gamma_0.3_theta_1e-13
runtime = 0.02 seconds
runtime = 0.16 seconds
runtime = 0.15 seconds
runtime = 0.10 seconds
runtime = 0.03 seconds
runtime = 0.04 seconds
runtime = 0.02 seconds
runtime = 0.02 seconds
runtime = 0

/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/envs/registration.py:498: UserWarning: WARN: Overriding environment MountainCarEnv-v1_cust already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


...

runtime = 0.07 seconds
runtime = 0.06 seconds
runtime = 0.09 seconds
runtime = 0.07 seconds
runtime = 0.01 seconds
runtime = 0.02 seconds
runtime = 0.02 seconds
runtime = 0.02 seconds
runtime = 0.01 seconds
here
./ADA/MC/Valu_iter/ VI_MC_size_20_gamma_0.01_theta_1e-13
runtime = 0.05 seconds
runtime = 0.10 seconds
runtime = 0.14 seconds
runtime = 0.05 seconds
runtime = 0.04 seconds
runtime = 0.04 seconds
runtime = 0.03 seconds
runtime = 0.04 seconds
runtime = 0.03 seconds
runtime = 0.03 seconds
here
./ADA/MC/Valu_iter/ VI_MC_size_20_gamma_0.1_theta_1e-13
runtime = 0.09 seconds
runtime = 0.06 seconds
runtime = 0.09 seconds
runtime = 0.28 seconds
runtime = 0.08 seconds
runtime = 0.05 seconds
runtime = 0.06 seconds
runtime = 0.04 seconds
runtime = 0.04 seconds
runtime = 0.05 seconds
here
./ADA/MC/Valu_iter/ VI_MC_size_20_gamma_0.3_theta_1e-13
runtime = 0.43 seconds
runtime = 0.28 seconds
runtime = 0.13 seconds
runtime = 0.13 seconds
runtime = 0.10 seconds
runtime = 0.12 seconds
runtime = 0

/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/envs/registration.py:498: UserWarning: WARN: Overriding environment MountainCarEnv-v1_cust already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


runtime = 0.03 seconds
runtime = 0.20 seconds
runtime = 0.08 seconds
runtime = 0.04 seconds
runtime = 0.03 seconds
runtime = 0.03 seconds
runtime = 0.02 seconds
runtime = 0.03 seconds
runtime = 0.02 seconds
runtime = 0.02 seconds
here
./ADA/MC/Valu_iter/ VI_MC_size_25_gamma_0.01_theta_1e-13
runtime = 0.24 seconds
runtime = 0.09 seconds
runtime = 0.08 seconds
runtime = 0.05 seconds
runtime = 0.06 seconds
runtime = 0.09 seconds
runtime = 0.05 seconds
runtime = 0.04 seconds
runtime = 0.04 seconds
runtime = 0.04 seconds
here
./ADA/MC/Valu_iter/ VI_MC_size_25_gamma_0.1_theta_1e-13
runtime = 0.33 seconds
runtime = 0.11 seconds
runtime = 0.12 seconds
runtime = 0.14 seconds
runtime = 0.21 seconds
runtime = 0.08 seconds
runtime = 0.11 seconds
runtime = 0.10 seconds
runtime = 0.08 seconds
runtime = 0.11 seconds
here
./ADA/MC/Valu_iter/ VI_MC_size_25_gamma_0.3_theta_1e-13
runtime = 0.34 seconds
runtime = 0.49 seconds
runtime = 0.20 seconds
runtime = 0.19 seconds
runtime = 0.20 seconds
runtime = 0

/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/envs/registration.py:498: UserWarning: WARN: Overriding environment MountainCarEnv-v1_cust already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


runtime = 0.06 seconds
runtime = 0.14 seconds
runtime = 0.24 seconds
runtime = 0.05 seconds
runtime = 0.05 seconds
runtime = 0.07 seconds
runtime = 0.05 seconds
runtime = 0.04 seconds
runtime = 0.04 seconds
here
./ADA/MC/Valu_iter/ VI_MC_size_30_gamma_0.01_theta_1e-13
runtime = 0.21 seconds
runtime = 0.18 seconds
runtime = 0.09 seconds
runtime = 0.33 seconds
runtime = 0.26 seconds
runtime = 0.09 seconds
runtime = 0.08 seconds
runtime = 0.08 seconds
runtime = 0.07 seconds
runtime = 0.07 seconds
here
./ADA/MC/Valu_iter/ VI_MC_size_30_gamma_0.1_theta_1e-13
runtime = 0.18 seconds
runtime = 0.49 seconds
runtime = 0.17 seconds
runtime = 0.21 seconds
runtime = 0.19 seconds
runtime = 0.18 seconds
runtime = 0.13 seconds
runtime = 0.13 seconds
runtime = 0.23 seconds
runtime = 0.15 seconds
here
./ADA/MC/Valu_iter/ VI_MC_size_30_gamma_0.3_theta_1e-13
runtime = 0.46 seconds
runtime = 0.50 seconds
runtime = 0.46 seconds
runtime = 0.53 seconds
runtime = 0.40 seconds
runtime = 0.34 seconds
runtime = 0

## Policy Iterations | all sizes | mountain car 

In [34]:

sizes_rid = [5,10,15,20,25,30]#5,#[8,12,16,20]#[4,8,12,16,20]

grid = {

'gamma' : [0.01,0.1,0.3,0.5,0.7,0.9,0.95,0.99],#1],
'theta' : [ 1e-13 ]


    
    
}

misc = 'PI_MC'
vi = False


env_name ='MC'
iter_ = 'Polic_iter'




PROCESSES=8
type_ = 'starmap_async'
n_iters=1000#00
gym_configurations = {'id' : 'MountainCarEnv-v1_cust','render_mode':None}
map_seed = 1
count =5#10

version = '2'



base =False
max_episode_steps=2000





generate_random_map_config = dict(p =0.9)
multiplier = 1000





#map_list =[], env_list = [],

create_new_map_flag = True




resp_ = run_VI_PI_model_over_grid_mc(grid =grid
                              ,sizes_rid =sizes_rid
                              ,misc =misc
                              ,vi=vi
                              ,env_name =env_name,iter_ = iter_,
                             version = version
                              
                              ,PROCESSES = PROCESSES
                                ,type_ = type_
                                ,n_iters = n_iters
                                ,gym_configurations = gym_configurations
                                ,map_seed = map_seed
                                ,count = count
                                ,generate_random_map_config = generate_random_map_config
                                ,multiplier = multiplier
                             ,create_new_map_flag = create_new_map_flag
                              ,base = base
                                ,max_episode_steps = max_episode_steps
                              
                             )





/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/envs/registration.py:498: UserWarning: WARN: Overriding environment MountainCarEnv-v1_cust already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


runtime = 0.00 seconds
runtime = 0.02 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
here
./ADA/MC/Polic_iter/ PI_MC_size_5_gamma_0.01_theta_1e-13
runtime = 0.01 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
here
./ADA/MC/Polic_iter/ PI_MC_size_5_gamma_0.1_theta_1e-13
runtime = 0.00 seconds
runtime = 0.02 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.02 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
here
./ADA/MC/Polic_iter/ PI_MC_size_5_gamma_0.3_theta_1e-13
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.02 seconds
runtime = 0.00 seconds
runtime = 0

/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/envs/registration.py:498: UserWarning: WARN: Overriding environment MountainCarEnv-v1_cust already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.09 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
here
./ADA/MC/Polic_iter/ PI_MC_size_10_gamma_0.01_theta_1e-13
runtime = 0.00 seconds
runtime = 0.01 seconds
runtime = 0.09 seconds
runtime = 0.06 seconds
runtime = 0.04 seconds
runtime = 0.01 seconds
runtime = 0.01 seconds
runtime = 0.00 seconds
runtime = 0.01 seconds
here
./ADA/MC/Polic_iter/ PI_MC_size_10_gamma_0.1_theta_1e-13
runtime = 0.04 seconds
runtime = 0.07 seconds
runtime = 0.06 seconds
runtime = 0.09 seconds
runtime = 0.11 seconds
runtime = 0.01 seconds
runtime = 0.01 seconds
runtime = 0.01 seconds
runtime = 0.01 seconds
here
./ADA/MC/Polic_iter/ PI_MC_size_10_gamma_0.3_theta_1e-13
runtime = 0.12 seconds
runtime = 0.10 seconds
runtime = 0.04 seconds
runtime = 0.01 seconds
runtime = 0.03 seconds
runtime = 0.03 seconds
runtime = 0.01 seconds
runtime = 0.01 seconds
runtime 

/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/envs/registration.py:498: UserWarning: WARN: Overriding environment MountainCarEnv-v1_cust already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


...

runtime = 0.06 seconds
runtime = 0.03 seconds
runtime = 0.04 seconds
runtime = 0.04 seconds
runtime = 0.05 seconds
runtime = 0.11 seconds
runtime = 0.02 seconds
runtime = 0.04 seconds
runtime = 0.02 seconds
here
./ADA/MC/Polic_iter/ PI_MC_size_20_gamma_0.01_theta_1e-13
runtime = 0.05 seconds
runtime = 0.05 seconds
runtime = 0.05 seconds
runtime = 0.33 seconds
runtime = 0.10 seconds
runtime = 0.05 seconds
runtime = 0.04 seconds
runtime = 0.05 seconds
runtime = 0.04 seconds
here
./ADA/MC/Polic_iter/ PI_MC_size_20_gamma_0.1_theta_1e-13
runtime = 0.07 seconds
runtime = 0.34 seconds
runtime = 0.15 seconds
runtime = 0.11 seconds
runtime = 0.26 seconds
runtime = 0.06 seconds
runtime = 0.07 seconds
runtime = 0.08 seconds
runtime = 0.07 seconds
here
./ADA/MC/Polic_iter/ PI_MC_size_20_gamma_0.3_theta_1e-13
runtime = 0.29 seconds
runtime = 0.36 seconds
runtime = 0.25 seconds
runtime = 0.49 seconds
runtime = 0.39 seconds
runtime = 0.15 seconds
runtime = 0.13 seconds
runtime = 0.13 seconds
runtime 

/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/envs/registration.py:498: UserWarning: WARN: Overriding environment MountainCarEnv-v1_cust already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


runtime = 0.17 seconds
runtime = 0.18 seconds
runtime = 0.18 seconds
runtime = 0.17 seconds
runtime = 0.08 seconds
runtime = 0.04 seconds
runtime = 0.04 seconds
runtime = 0.03 seconds
runtime = 0.09 seconds
here
./ADA/MC/Polic_iter/ PI_MC_size_25_gamma_0.01_theta_1e-13
runtime = 0.22 seconds
runtime = 0.10 seconds
runtime = 0.10 seconds
runtime = 0.14 seconds
runtime = 0.12 seconds
runtime = 0.15 seconds
runtime = 0.10 seconds
runtime = 0.09 seconds
runtime = 0.12 seconds
here
./ADA/MC/Polic_iter/ PI_MC_size_25_gamma_0.1_theta_1e-13
runtime = 0.35 seconds
runtime = 0.36 seconds
runtime = 0.16 seconds
runtime = 0.19 seconds
runtime = 0.18 seconds
runtime = 0.41 seconds
runtime = 0.14 seconds
runtime = 0.14 seconds
runtime = 0.17 seconds
here
./ADA/MC/Polic_iter/ PI_MC_size_25_gamma_0.3_theta_1e-13
runtime = 0.30 seconds
runtime = 0.23 seconds
runtime = 0.23 seconds
runtime = 0.42 seconds
runtime = 0.25 seconds
runtime = 0.22 seconds
runtime = 0.60 seconds
runtime = 0.24 seconds
runtime 

/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/envs/registration.py:498: UserWarning: WARN: Overriding environment MountainCarEnv-v1_cust already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


runtime = 0.13 seconds
runtime = 0.26 seconds
runtime = 0.30 seconds
runtime = 0.24 seconds
runtime = 0.10 seconds
runtime = 0.06 seconds
runtime = 0.06 seconds
runtime = 0.07 seconds
runtime = 0.06 seconds
runtime = 0.06 seconds
here
./ADA/MC/Polic_iter/ PI_MC_size_30_gamma_0.01_theta_1e-13
runtime = 0.19 seconds
runtime = 0.43 seconds
runtime = 0.27 seconds
runtime = 0.15 seconds
runtime = 0.32 seconds
runtime = 0.12 seconds
runtime = 0.14 seconds
runtime = 0.17 seconds
runtime = 0.14 seconds
runtime = 0.11 seconds
here
./ADA/MC/Polic_iter/ PI_MC_size_30_gamma_0.1_theta_1e-13
runtime = 0.17 seconds
runtime = 0.31 seconds
runtime = 0.61 seconds
runtime = 0.24 seconds
runtime = 0.24 seconds
runtime = 0.28 seconds
runtime = 0.19 seconds
runtime = 0.20 seconds
here
./ADA/MC/Polic_iter/ PI_MC_size_30_gamma_0.3_theta_1e-13
runtime = 0.36 seconds
runtime = 0.34 seconds
runtime = 0.42 seconds
runtime = 0.33 seconds
runtime = 0.75 seconds
runtime = 0.30 seconds
runtime = 0.31 seconds
runtime 